# Emulated Distributed Tridiagonal Solver

### Creating the system

The tridiagonal system we want to solve is one with constant coefficients; first, we create it:

In [19]:
import numpy as np

In [20]:
system_size = 20
a = np.ones(system_size)*(1./4)
b = np.ones(system_size)
c = np.ones(system_size)*(1./4)
a[-1] = 2
c[0] = 2
rhs = np.random.rand(system_size)

### Solving the system with SciPy

We solve it using SciPy's `solve_banded`:

In [21]:
from scipy.linalg import solve_banded

# an interface to SciPy's tridiagonal
# solver - used for testing
def scipy_solve_banded(a, b, c, rhs):
    '''
    Solve the tridiagonal system described
    by a, b, c, and rhs.
    a: lower off-diagonal array (first element ignored)
    b: diagonal array
    c: upper off-diagonal array (last element ignored)
    rhs: right hand side of the system
    '''
    l_and_u = (1, 1)
    ab = np.vstack([np.append(0, c[:-1]),
                    b,
                    np.append(a[1:], 0)])
    x = solve_banded(l_and_u, ab, rhs)
    return x


In [22]:
x_scipy = scipy_solve_banded(a, b, c, rhs)

### A Standard LU Solver


In [23]:
def LU_tridag(a, b, c, r, u):
    gam = np.zeros_like(a)
    N = a.size
    
    beta = 1./b[0]
    u[0] = beta*r[0]
    
    for j in range(1, N):
        gam[j] = beta*c[j-1]
        beta = 1./(b[j] - a[j]*gam[j])
        u[j] = beta*(r[j] - a[j]*u[j-1])
    
    for j in range(N-2, -1, -1):
        u[j] = u[j] - gam[j+1]*u[j+1]
    
    return


In [24]:
x_LU = np.zeros_like(a)
LU_tridag(a, b, c, rhs, x_LU)

In [25]:
from numpy.testing import assert_allclose
assert_allclose(x_LU, x_scipy)

### Pre-computing $\beta$

In [26]:
def precompute_beta(a, b, c, beta):
    # pre-computes the beta required
    # by the tridiagonal solver
    
    N = a.size
    beta[0] = 1./b[0]
    
    for j in range(1, N):
        beta[j] = 1./(b[j] - a[j]*beta[j-1]*c[j-1])
    
    return

def LU_tridag_precomputed_beta(a, b, c, r, beta, u):
    # The standard LU solver with
    # beta pre-computed:

    gam = np.zeros_like(a)
    N = a.size
    
    u[0] = beta[0]*r[0]
    
    for j in range(1, N):
        gam[j] = beta[j-1]*c[j-1]
        u[j] = beta[j]*(r[j] - a[j]*u[j-1])
    
    
    for j in range(N-2, -1, -1):
        u[j] = u[j] - gam[j+1]*u[j+1]

    return
    

In [27]:
x_LU_precomputed_beta = np.zeros_like(a, dtype=np.float64)
beta = np.zeros_like(a, dtype=np.float64)
precompute_beta(a, b, c, beta)
LU_tridag_precomputed_beta(a, b, c, rhs, beta, x_LU_precomputed_beta)

In [28]:
assert_allclose(x_LU_precomputed_beta, x_scipy)

### Pre-computing both $\beta$ and $\gamma$:

In [29]:
def precompute_beta_gam(a, b, c, beta, gam):
    # pre-computes the beta and gam required
    # by the tridiagonal solver
    
    N = a.size
    beta[0] = 1./b[0]
    gam[0] = 0.0
    
    for j in range(1, N):
        beta[j] = 1./(b[j] - a[j]*beta[j-1]*c[j-1])
        gam[j] = beta[j-1]*c[j-1]
    
    return
    

def LU_tridag_precomputed_beta_gam(a, b, c, r, beta, gam, u):
    # The standard LU solver with
    # beta and gam pre-computed:

    N = a.size
    
    u[0] = beta[0]*r[0]
    
    for j in range(1, N):
        u[j] = beta[j]*(r[j] - a[j]*u[j-1])
        
    for j in range(N-2, -1, -1):
        u[j] = u[j] - gam[j+1]*u[j+1]
    
    return
    

In [30]:
x_LU_precomputed_beta_gam = np.zeros_like(a, dtype=np.float64)
beta = np.zeros_like(a, dtype=np.float64)
gam = np.zeros_like(a, dtype=np.float64)
precompute_beta_gam(a, b, c, beta, gam)
LU_tridag_precomputed_beta_gam(a, b, c, rhs, beta, gam, x_LU_precomputed_beta_gam)

In [31]:
assert_allclose(x_LU_precomputed_beta_gam, x_scipy)

### An emulated distributed solver

In [38]:
def tridag_edits(a, b, c, r, beta, gam, u, nprocs):
    #############
    # L-R sweep
    #############
    system_size = a.size
    assert(system_size%nprocs == 0)

    local_size = system_size/nprocs
    
    phi = np.zeros_like(a, dtype=np.float64)
    psi = np.zeros_like(a, dtype=np.float64)
    
    # each "processor" computes its phi and psi
    for proc, start in enumerate(range(0, system_size, local_size)):
        if proc == 0:
            phi[start+0] = 0
            psi[start+0] = 1
        else:
            phi[start+0] = beta[start+0]*r[start+0]
            psi[start+0] = -a[1]*beta[start+0]
        
        for i in range(1, local_size):
            phi[start+i] = beta[start+i]*(r[start+i] - a[1]*phi[start+i-1])
            psi[start+i] = -a[1]*beta[start+i]*psi[start+i-1]
        if proc == nprocs-1:
            phi[start+i] = beta[start+i]*(r[start+i] - a[-1]*phi[start+i-1])
            psi[start+local_size-1] = -a[-1]*beta[start+i]*psi[start+i-1]
        
    
    # each "processor" posts its last phi and psi
    phi_lasts = phi[local_size-1::local_size]
    psi_lasts = psi[local_size-1::local_size]
    
    # each "processor" uses the last phi and psi from the 
    # previous "processors" to compute its u_tilda:
    # the first "processor" just uses u_0
    u_tildas = np.zeros(nprocs, dtype=np.float64)
    u[0] = beta[0]*r[0]
    for proc, start in enumerate(range(0, system_size, local_size)):
        if proc == 0:
            u_tildas[proc] = u[0]
        else:
            product_2 = 1.0
            for i in range(proc):
                # compute the product terms:
                product_1 = 1.0 
                for j in range(i+1, proc):
                    product_1 *= psi_lasts[j]
                u_tildas[proc] += phi_lasts[i]*product_1
                product_2 *= psi_lasts[i]
                
            u_tildas[proc] += u[0]*product_2
            
    
    # Now that each processor has
    # its u_tilda,
    # the entire u can be computed (in parallel, in fact):    
    for proc, start in enumerate(range(0, system_size, local_size)):
        u[start] = u_tildas[proc]
        for i in range(local_size):
            u[start+i] = phi[start+i] + psi[start+i]*u_tildas[proc]
    
    #############
    # R-L sweep
    #############
    
    # each "processor" will need the first 'gam' from the next processor:
    gam_firsts = gam[0::local_size]
    
    # each "processor" computes its phi and psi
    for proc, end in reversed(list(enumerate(range(local_size-1, system_size, local_size)))):
        if proc == nprocs-1:
            phi[end-0] = 0
            psi[end-0] = 1
        else:
            phi[end-0] = u[end-0]
            psi[end-0] = -gam_firsts[proc+1]
            
        for i in range(1, local_size):
            phi[end-i] = u[end-i] - gam[end-i+1]*phi[end-i+1]
            psi[end-i] = -gam[end-i+1]*psi[end-i+1]
    
    # each "processor" posts its first phi and psi
    phi_firsts = phi[0::local_size]
    psi_firsts = psi[0::local_size]
    
    # each "processor" uses the first phi and psi from the 
    # next "processors" to compute its x_tilda:
    # the last "processor" just uses x_(n-1)
    x_tildas = np.zeros(nprocs, dtype=np.float64)
    x = np.zeros_like(u, dtype=np.float64)
    x[-1] = u[-1]
    for proc, end in reversed(list(enumerate(range(local_size-1, system_size, local_size)))):
        if proc == nprocs-1:
            x_tildas[proc] = x[-1]
        else:
            for i in range(proc+2, nprocs):
                product_1 = 1.0
                for j in range(proc+1, i):
                    product_1 *= psi_firsts[j]
                x_tildas[proc] += phi_firsts[i]*product_1
            
            product_2 = 1.0    
            for i in range(proc+1, nprocs):
                product_2 *= psi_firsts[i]
                
            x_tildas[proc] += phi_firsts[proc+1] + x[-1]*product_2
    
    # Now that each processor has
    # its x_tilda,
    # the entire x can be computed (in parallel, in fact):
    for proc, end in reversed(list(enumerate(range(local_size-1, system_size, local_size)))):
        for i in range(local_size):
            x[end-i] = phi[end-i] + x_tildas[proc]*psi[end-i]
    return x
    

In [39]:
x_edits = np.zeros_like(a, dtype=np.float64)
beta = np.zeros_like(a, dtype=np.float64)
gam = np.zeros_like(a, dtype=np.float64)
precompute_beta_gam(a, b, c, beta, gam)
x = tridag_edits(a, b, c, rhs, beta, gam, x_edits, 4)
assert_allclose(x_scipy, x)

NameError: name 'plt' is not defined